In [1]:
import os 
import tqdm
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../data/'

In [2]:
train = np.loadtxt(os.path.join(dir_, 'rating_matrix.csv'), delimiter=',')
pd_train = pd.read_pickle(os.path.join(dir_, 'train_normalized_to_rating_filter_track_5_user_100.pkl'))
train

array([[0.  , 2.5 , 0.25, ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       ...,
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ]])

In [3]:
num_user = len(train)
num_track = len(train[0])
print(num_user, num_track)

220 54969


In [4]:
def pearson_correlation(u1_index, u2_index):
        result = 0.0
        user1_data = train[u1_index]
        user2_data = train[u2_index]

        rx_avg = user_average_rating(user1_data)
        ry_avg = user_average_rating(user2_data)
        sxy = common_items(user1_data, user2_data)

        top_result = 0.0
        bottom_left_result = 0.0
        bottom_right_result = 0.0
        for item in sxy:
            rxs = user1_data[item]
            rys = user2_data[item]

            top_result += (rxs - rx_avg)*(rys - ry_avg)
            bottom_left_result += pow((rxs - rx_avg), 2)
            bottom_right_result += pow((rys - ry_avg), 2)
        bottom_left_result = math.sqrt(bottom_left_result)
        bottom_right_result = math.sqrt(bottom_right_result)
        
        ################################################################
        if (bottom_left_result * bottom_right_result) == 0:
            return -2, -2 # dump the data
        ################################################################
        
        result = top_result/(bottom_left_result * bottom_right_result)
        return len(sxy), result

def user_average_rating(u):
    avg_rating = 0.0
    for i in u:
        avg_rating += i
    avg_rating /= len(u) * 1.0
    return avg_rating

def common_items(u1, u2):
    result = []
    for i in range(num_track):
        if u1[i] > 0 and u2[i] > 0:
            result.append(i)
    return result

In [7]:
sim_table = pd.DataFrame(columns=['u1', 'u2', 'common', 'pearson']) 
for i in tqdm(range(num_user)):
    for j in range(i+1, num_user):
        c, p = pearson_correlation(i,j)
        if p != -2:
            p = p * min(c,200) / 200
            sim_table = sim_table.append({'u1': i, 'u2': j, 'common': c, 'pearson': p}, ignore_index=True)
            sim_table = sim_table.append({'u1': j, 'u2': i, 'common': c, 'pearson': p}, ignore_index=True)

In [38]:
# sim_table_ = sim_table[sim_table['common']>10]
sim_table_ = sim_table.copy()
sim_table_ = sim_table_.sort_values(by=['pearson'],  ascending=False)
sim_table_ = sim_table_.reset_index()
high_similarity = sim_table_[:int(len(sim_table_)*2/10)]
low_similarity =sim_table_[int(len(sim_table_)*8/10):]
high_similarity_v = sim_table_.iloc[int(len(sim_table_)*2/10)]['pearson']
low_similarity_v = sim_table_.iloc[int(len(sim_table_)*8/10)]['pearson']
print(high_similarity_v, low_similarity_v)

0.34935574741110736 0.02704130939761595


In [39]:
pairs = pd.DataFrame(columns=['uid', 'pair']) 
for i in range(num_user):
    pair = len(high_similarity[high_similarity['u1'] == i])
    pairs = pairs.append({'uid': i, 'pair': pair}, ignore_index=True)
pairs = pairs.sort_values(by=['pair'],  ascending=True)

groups = []
group_size = 2
pairs = pairs[pairs['pair'] >= (group_size-1)]
p_status = len(pairs)
pbar = tqdm(total=len(pairs)) 
while len(pairs) >= group_size:
    group = []
    uid = pairs.iloc[0]['uid']
    pbar.update(p_status - len(pairs))
    p_status -= (p_status - len(pairs))
    pair = pairs.iloc[0]['pair']
    group.append(uid)
    if pair == (group_size-1):
        uid = high_similarity[high_similarity['u1'] == uid]
        for _, row in uid.iterrows():
            group.append(int(row['u2']))
        check_sim = True
        for i in range(1, group_size):
            for j in range(i+1, group_size):
                check = high_similarity[high_similarity['u1'] == group[i]]
                check = check[check['u2'] == group[j]]
                if len(check) == 0:
                    check_sim = False
        if check_sim:
#             print(group)
            groups.append(group)
            for u in group:
                high_similarity = high_similarity[high_similarity['u1'] != u]
                high_similarity = high_similarity[high_similarity['u2'] != u]
        else:
            high_similarity = high_similarity[high_similarity['u1'] != group[0]]
            high_similarity = high_similarity[high_similarity['u2'] != group[0]]
        pairs = pd.DataFrame(columns=['uid', 'pair']) 
        for i in range(num_user):
            pair = len(high_similarity[high_similarity['u1'] == i])
            pairs = pairs.append({'uid': i, 'pair': pair}, ignore_index=True)
        pairs = pairs.sort_values(by=['pair'],  ascending=True)
        pairs = pairs[pairs['pair'] >= (group_size-1)]
    else:
        uid = high_similarity[high_similarity['u1'] == uid]
        avaliable_uid = []
        for _, row in pairs.iterrows():
            if len(uid[uid['u2'] == row['uid']]) > 0:
                avaliable_uid.append(int(row['uid']))
                
        pointer = []
        for i in range(group_size-1):
            pointer.append(i)

        while pointer[0] < (len(avaliable_uid) - (group_size - 1) + 2) and len(avaliable_uid) >= (group_size-1):
            group_ = group.copy()
            for i in range(group_size-1):
                group_.append(avaliable_uid[pointer[i]])
            check_sim = True
            for i in range(1, group_size):
                for j in range(i+1, group_size):
                    check = high_similarity[high_similarity['u1'] == group_[i]]
                    check = check[check['u2'] == group_[j]]
                    if len(check) == 0:
                        check_sim = False
            if check_sim:
#                 print('_', group_)
                groups.append(group_)
                for u in group_:
                    high_similarity = high_similarity[high_similarity['u1'] != u]
                    high_similarity = high_similarity[high_similarity['u2'] != u]
                break
            else:
                p = 0
                added = False
                while not added:
                    if (group_size-2) - p < 0:
                        pointer[0] = (len(avaliable_uid) - (group_size - 1) + 2)
                        break
                    if pointer[(group_size-2) - p] < (len(avaliable_uid) - 1 - p):
                        pointer[(group_size-2) - p] += 1
                        while p > 0:
                            p -= 1
                            if (group_size-2) - p != 0:
                                pointer[(group_size-2) - p] = pointer[(group_size-2) - p - 1] + 1
                        added = True
                    else:
                        p += 1
        if pointer[0] == (len(avaliable_uid) - (group_size - 1) + 2) or len(avaliable_uid) < (group_size-1):
            high_similarity = high_similarity[high_similarity['u1'] != group[0]]
            high_similarity = high_similarity[high_similarity['u2'] != group[0]]
        pairs = pd.DataFrame(columns=['uid', 'pair']) 
        for i in range(num_user):
            pair = len(high_similarity[high_similarity['u1']==i])
            pairs = pairs.append({'uid': i, 'pair': pair}, ignore_index=True)
        pairs = pairs.sort_values(by=['pair'],  ascending=True)
        pairs = pairs[pairs['pair'] >= (group_size-1)]
groups

[[21, 178],
 [67, 182],
 [136, 72],
 [156, 148],
 [208, 169],
 [163, 73],
 [215, 116],
 [2, 189],
 [200, 188],
 [159, 167],
 [49, 151],
 [112, 50],
 [54, 196],
 [166, 33],
 [149, 74],
 [96, 41],
 [62, 7],
 [78, 115],
 [205, 52],
 [87, 207],
 [184, 134],
 [28, 66],
 [91, 3],
 [161, 88],
 [76, 147],
 [141, 45],
 [109, 211],
 [48, 210],
 [8, 19],
 [203, 79],
 [144, 130],
 [108, 124],
 [187, 22],
 [98, 13],
 [192, 174],
 [11, 105],
 [102, 100],
 [32, 165],
 [37, 164],
 [106, 213],
 [158, 140],
 [97, 132],
 [95, 75],
 [56, 185],
 [212, 55],
 [120, 31],
 [101, 127],
 [157, 219],
 [135, 155],
 [17, 103],
 [150, 80],
 [20, 43],
 [190, 194],
 [89, 71],
 [63, 191],
 [214, 126],
 [14, 142],
 [46, 198],
 [99, 111],
 [176, 123],
 [160, 180],
 [125, 59],
 [90, 81],
 [64, 93],
 [137, 119],
 [38, 39],
 [195, 138],
 [183, 68],
 [170, 77],
 [24, 6],
 [40, 47],
 [23, 177],
 [173, 209],
 [10, 82],
 [199, 113],
 [58, 0],
 [143, 193],
 [60, 171],
 [12, 172],
 [110, 57],
 [202, 16],
 [179, 61],
 [4, 107]]

In [40]:
for group in groups:
    if len(group) != group_size:
        print('Group size error')
    for i in range(group_size):
        for j in range(i+1, group_size):
            u1 = group[i]
            u2 = group[j]
            c, p = pearson_correlation(u1, u2)
            if p < high_similarity_v:
                print('Similarity error', u1, u2, p)
user_status = np.zeros(num_user)
for group in groups:
    for u in group:
        user_status[u] += 1
for i in user_status:
    if i > 1:
        print('Depulicate error')

In [41]:
len(groups)

83

In [42]:
np.savetxt(os.path.join(dir_, 'h_group_' + str(group_size)  + '.csv'), groups, delimiter=",")